In [1]:
# coding: utf8

In [2]:
import math
import random
import pandas as pd
import numpy  as np
import seaborn as sb
import matplotlib.pyplot as plt

In [3]:
from __future__ import division
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import train_test_split

In [4]:
import warnings
%matplotlib inline
warnings.filterwarnings("ignore")
sb.set(style="white", color_codes=True)

In [5]:
SEED_STATE = 1
KFOLDS_NUM = 8

In [6]:
claims = pd.read_csv('train.csv')

In [7]:
claims.head()

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,...,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Z,NaN,NaN,NaN,0,NaN,NaN


In [8]:
all_features = claims.drop(["ID", "target"] ,axis=1).columns
dict_types = all_features.to_series().groupby(claims.dtypes).groups
dict_types = {k.name: v for k, v in dict_types.items()}
print "Here the differents types columns in the df: ",dict_types.keys()
int_columns = dict_types[u'int64']
cat_columns = dict_types[u'object']
float_columns = dict_types[u'float64']
print "Number of interger columns: \t ",len(int_columns)
print "Number of letters  columns: \t ",len(cat_columns)

Here the differents types columns in the df:  ['object', 'int64', 'float64']
Number of interger columns: 	  4
Number of letters  columns: 	  19


In [9]:
columns = cat_columns

In [10]:
train = claims[["ID", "target"]]

In [11]:
#note 56 et 125 et 22

In [12]:
for col in columns:
    if col in ["v125", "v56", "v22", "v113", "v112"]: 
        continue
    print "col: %s = %s "%(col,claims[col].unique())

col: v3 = ['C' nan 'A' 'B'] 
col: v24 = ['C' 'E' 'D' 'A' 'B'] 
col: v30 = ['C' nan 'G' 'D' 'B' 'F' 'A' 'E'] 
col: v31 = ['A' 'B' nan 'C'] 
col: v47 = ['C' 'E' 'I' 'D' 'F' 'G' 'J' 'B' 'A' 'H'] 
col: v52 = ['G' 'F' 'H' 'K' 'A' 'C' 'E' 'J' 'D' 'I' 'L' 'B' nan] 
col: v66 = ['C' 'A' 'B'] 
col: v71 = ['F' 'B' 'C' 'L' 'I' 'A' 'G' 'K' 'D'] 
col: v74 = ['B' 'C' 'A'] 
col: v75 = ['D' 'B' 'C' 'A'] 
col: v79 = ['E' 'D' 'B' 'C' 'I' 'P' 'A' 'O' 'K' 'M' 'J' 'H' 'F' 'Q' 'R' 'N' 'G' 'L'] 
col: v91 = ['A' 'B' 'G' 'C' 'F' 'E' 'D' nan] 
col: v107 = ['E' 'B' 'C' 'D' 'A' 'F' 'G' nan] 
col: v110 = ['B' 'A' 'C'] 


In [13]:
for col in columns:
    if col in ["v125", "v56", "v22", "v113", "v112"]:
        continue
    for val in claims[col].unique():
        if isinstance(val, float) and math.isnan(val):
            continue
        train["%s.%s"%(col,val)] = map(lambda y: 1 if y == val else 0, claims[col])

In [14]:
print(len(train.columns.tolist()))

96


In [15]:
train.head()

,ID,target,v3.C,v3.A,v3.B,v24.C,v24.E,v24.D,v24.A,v24.B,...,v107.E,v107.B,v107.C,v107.D,v107.A,v107.F,v107.G,v110.B,v110.A,v110.C
0,3,1,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
1,4,1,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,5,1,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0
3,6,1,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
4,8,1,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [21]:
#train.drop(["target"], axis=1).to_csv("train_bin.csv", sep=",", index=False)

In [15]:
train_data, test_data = train_test_split(train, test_size=0.2, random_state=SEED_STATE)

In [16]:
forest = ExtraTreesClassifier(n_estimators=1200, max_features=30, criterion='entropy', min_samples_split= 2, max_depth= 30, 
min_samples_leaf= 2, n_jobs = -1)

In [17]:
forest.fit(train_data.drop(["ID", "target"], axis=1), train_data["target"])

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='entropy',
           max_depth=30, max_features=30, max_leaf_nodes=None,
           min_samples_leaf=2, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1200, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [18]:
forest.score(test_data.drop(["ID", "target"], axis=1), test_data["target"])

0.74541876230045923

In [17]:
test = pd.read_csv('test.csv')

In [18]:
result = test[["ID"]]
test = test[columns]

In [19]:
for col in columns:
    if col in ["v125", "v56", "v22", "v113", "v112"]:
        continue
    for val in claims[col].unique():
        if isinstance(val, float) and math.isnan(val):
            continue
        result["%s.%s"%(col,val)] = map(lambda y: 1 if y == val else 0, test[col])

In [20]:
result.head(5)

,ID,v3.C,v3.A,v3.B,v24.C,v24.E,v24.D,v24.A,v24.B,v30.C,...,v107.E,v107.B,v107.C,v107.D,v107.A,v107.F,v107.G,v110.B,v110.A,v110.C
0,0,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,1,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,0
2,2,1,0,0,0,0,0,0,1,1,...,0,0,0,1,0,0,0,0,1,0
3,7,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,10,1,0,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0


In [23]:
result.to_csv("test_bin.csv", sep=",", index=False)

In [23]:
#res = forest.predict(result.drop(["ID"], axis=1))

In [24]:
result["binary"] = forest.predict(result.drop(["ID"], axis=1))

In [25]:
result.to_csv("lmd_binary.csv", columns=["ID", "binary"], index=False)